In [2]:
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
from pathlib import Path

/home/jace/miniconda3/envs/climate-downscaling/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Create bounds
This notebook generates a geojson file from a csv of lat lon bounds and fixes geometries if in case they are not perfect rectangles

### Input parameters

In [3]:
INPUT_PATH = Path("../../data/01-raw/domains")

### Create geojson of domain bounds

In [3]:
df = pd.read_csv(INPUT_PATH / 'downscaling_domains.csv')
df

,city,lon0,lat0,lon1,lat1
0,Dagupan,119.6800,15.7290,120.6850,16.6520
1,Palayan,120.8469,15.2314,121.6681,15.9086
2,MetroManila,120.6903,14.2564,121.2177,14.0540
3,Legazpi,123.6415,13.0340,123.8365,13.2293
4,Iloilo,122.4124,10.5540,122.7393,10.8602
5,Mandaue,123.8338,10.2203,124.0418,10.4311
6,Tacloban,124.8637,11.3953,125.1480,11.0989
7,Zamboanga,121.7610,6.8060,122.5690,7.5860
8,CagayanDeOro,124.2220,8.8800,124.9690,8.1140
9,Davao,125.0130,6.6810,126.1010,7.7710


In [4]:
polygons = []

for index, row in df.iterrows():
    lat0, lat1, lon0, lon1 = row['lat0'], row['lat1'], row['lon0'], row['lon1']
    polygon = Polygon([(lon0, lat0), (lon1, lat0), (lon1, lat1), (lon0, lat1)])
    polygons.append(polygon)

df['geometry'] = polygons
df.index = pd.Index(df.index)
gdf = gpd.GeoDataFrame(df[['city','geometry']], geometry='geometry')
gdf

,city,geometry
0,Dagupan,"POLYGON ((119.68000 15.72900, 120.68500 15.729..."
1,Palayan,"POLYGON ((120.84690 15.23140, 121.66810 15.231..."
2,MetroManila,"POLYGON ((120.69030 14.25640, 121.21770 14.256..."
3,Legazpi,"POLYGON ((123.64150 13.03400, 123.83650 13.034..."
4,Iloilo,"POLYGON ((122.41240 10.55400, 122.73930 10.554..."
5,Mandaue,"POLYGON ((123.83380 10.22030, 124.04180 10.220..."
6,Tacloban,"POLYGON ((124.86370 11.39530, 125.14800 11.395..."
7,Zamboanga,"POLYGON ((121.76100 6.80600, 122.56900 6.80600..."
8,CagayanDeOro,"POLYGON ((124.22200 8.88000, 124.96900 8.88000..."
9,Davao,"POLYGON ((125.01300 6.68100, 126.10100 6.68100..."


In [5]:
gdf.to_file(INPUT_PATH / 'downscaling_domains.geojson', driver='GeoJSON')

/home/jace/miniconda3/envs/climate-downscaling/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


### Fix geometries to perfect rectangles

In [4]:
gdf = gpd.read_file(INPUT_PATH /  'downscaling_domains.geojson')
gdf

,city,geometry
0,Dagupan,"POLYGON ((120.00931 15.91018, 120.52218 15.907..."
1,Palayan,"POLYGON ((120.88251 15.29488, 121.33348 15.297..."
2,MetroManila,"POLYGON ((120.87219 14.79454, 121.20453 14.791..."
3,Legazpi,"POLYGON ((123.65317 12.95603, 123.88749 12.957..."
4,Iloilo,"POLYGON ((122.41374 10.59655, 122.69007 10.591..."
5,Mandaue,"POLYGON ((123.85434 10.22812, 124.06435 10.227..."
6,Tacloban,"POLYGON ((124.86370 11.39530, 125.11476 11.394..."
7,Zamboanga,"POLYGON ((121.85371 6.81269, 122.46680 6.81564..."
8,CagayanDeOro,"POLYGON ((124.34436 8.71236, 124.95302 8.71383..."
9,Davao,"POLYGON ((125.06060 6.84069, 125.83580 6.82890..."


In [6]:
perfect_rectangles = []
for index, row in gdf.iterrows():
    rough_polygon = row['geometry']
    min_x, min_y, max_x, max_y = rough_polygon.bounds
    perfect_polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])
    perfect_rectangles.append(perfect_polygon)
perfect_gdf = gpd.GeoDataFrame(gdf, geometry=perfect_rectangles, crs=gdf.crs)
perfect_gdf 

,city,geometry
0,Dagupan,"POLYGON ((120.00931 15.90723, 120.52218 15.907..."
1,Palayan,"POLYGON ((120.88251 15.29488, 121.33348 15.294..."
2,MetroManila,"POLYGON ((120.86704 14.29347, 121.20453 14.293..."
3,Legazpi,"POLYGON ((123.65317 12.95603, 123.88749 12.956..."
4,Iloilo,"POLYGON ((122.41374 10.59140, 122.69007 10.591..."
5,Mandaue,"POLYGON ((123.84991 10.22738, 124.06435 10.227..."
6,Tacloban,"POLYGON ((124.86370 11.11900, 125.11697 11.119..."
7,Zamboanga,"POLYGON ((121.85371 6.81269, 122.48448 6.81269..."
8,CagayanDeOro,"POLYGON ((124.34436 8.09633, 124.95891 8.09633..."
9,Davao,"POLYGON ((125.05176 6.82890, 125.83580 6.82890..."


In [7]:
perfect_gdf.to_file(INPUT_PATH / 'downscaling_domains_fixed.geojson', driver='GeoJSON')

/home/jace/miniconda3/envs/climate-downscaling/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
